In [1]:
import os
import cv2
from albumentations import Compose, HorizontalFlip, VerticalFlip, ShiftScaleRotate, RandomBrightnessContrast

/Users/valentinaolaritei/Desktop/3 SEMESTER/.venv/lib/python3.12/site-packages/albumentations/check_version.py:51: UserWarning: Error fetching version info <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>
  data = fetch_version_info()


In [2]:
# diverse augmentations

augmentation_type_1 = Compose([
    HorizontalFlip(p=0.5),
    ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7),
    RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5)
])

augmentation_type_2 = Compose([
    VerticalFlip(p=0.5),
    ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=45, p=0.7),
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, p=0.5)
])

augmentation_type_3 = Compose([
    HorizontalFlip(p=0.7),
    ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=60, p=0.5),
    RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.2, p=0.7)
])

In [3]:
# augmentations to apply

augmentations = [augmentation_type_1, augmentation_type_2, augmentation_type_3]

In [4]:
# directories

contrast_image_dir = 'dataset/images_with_contrast'
no_contrast_image_dir = 'dataset/images_no_contrast'
label_dir = 'dataset/labels'
augmented_contrast_image_dir = 'dataset/augmented_images_with_contrast'
augmented_no_contrast_image_dir = 'dataset/augmented_images_no_contrast'
augmented_label_dir = 'dataset/augmented_labels'

# create them
os.makedirs(augmented_contrast_image_dir, exist_ok=True)
os.makedirs(augmented_no_contrast_image_dir, exist_ok=True)
os.makedirs(augmented_label_dir, exist_ok=True)

In [5]:
def augment_image(image_path, label_path, output_image_dir, output_label_dir, augmentations):
    """
    Applies multiple augmentation pipelines to each image and saves results
    """
    
    print(f"Processing image: {image_path}")
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error reading image: {image_path}")
        return

    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Read the corresponding label file
    try:
        with open(label_path, 'r') as file:
            label_content = file.read()
    except FileNotFoundError:
        print(f"Label file not found for image: {image_path}")
        return
    
    # Apply each augmentation pipeline
    for aug_idx, augmentation in enumerate(augmentations):
        augmented = augmentation(image=image)['image']
        
        # Save augmented image
        aug_image_name = f"{base_name}_aug_{aug_idx}.png"
        aug_image_path = os.path.join(output_image_dir, aug_image_name)
        cv2.imwrite(aug_image_path, augmented)
        print(f"Saved augmented image: {aug_image_path}")
        
        # Save corresponding annotation file
        aug_label_name = f"{base_name}_aug_{aug_idx}.txt"
        aug_label_path = os.path.join(output_label_dir, aug_label_name)
        with open(aug_label_path, 'w') as label_file:
            label_file.write(label_content)
        print(f"Saved augmented label: {aug_label_path}")

def contains_target_class(label_path):
    """Check if the label file contains 'Non-Living' or 'Bubble' classes."""
    target_classes = {'1', '2'}  # Assuming 1 is 'Non-Living' and 2 is 'Bubble'
    with open(label_path, 'r') as file:
        for line in file:
            class_id = line.split()[0]  # Extract class_id from the annotation line
            if class_id in target_classes:
                return True
    return False


In [6]:
# Loop through images in both contrast and no-contrast folders
for folder, output_image_dir in [
    (contrast_image_dir, augmented_contrast_image_dir),
    (no_contrast_image_dir, augmented_no_contrast_image_dir)
]:
    print(f"Processing folder: {folder}")
    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            image_path = os.path.join(folder, filename)
            label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + '.txt')
            
            # Apply augmentation only if the label contains 'Non-Living' or 'Bubble' classes
            if contains_target_class(label_path):
                augment_image(image_path, label_path, output_image_dir, augmented_label_dir, augmentations)


Processing folder: dataset/images_with_contrast
Processing image: dataset/images_with_contrast/009-9752_frame_78.png
Saved augmented image: dataset/augmented_images_with_contrast/009-9752_frame_78_aug_0.png
Saved augmented label: dataset/augmented_labels/009-9752_frame_78_aug_0.txt
Saved augmented image: dataset/augmented_images_with_contrast/009-9752_frame_78_aug_1.png
Saved augmented label: dataset/augmented_labels/009-9752_frame_78_aug_1.txt
Saved augmented image: dataset/augmented_images_with_contrast/009-9752_frame_78_aug_2.png
Saved augmented label: dataset/augmented_labels/009-9752_frame_78_aug_2.txt
Processing image: dataset/images_with_contrast/009-9654_frame_90.png
Saved augmented image: dataset/augmented_images_with_contrast/009-9654_frame_90_aug_0.png
Saved augmented label: dataset/augmented_labels/009-9654_frame_90_aug_0.txt
Saved augmented image: dataset/augmented_images_with_contrast/009-9654_frame_90_aug_1.png
Saved augmented label: dataset/augmented_labels/009-9654_fra